In [35]:
!pip install arxiv
!pip install wikipedia


In [36]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper,verbose=True) 


In [37]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.langchain.com/labs/oap/quickstart")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectorstore = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vectorstore.as_retriever()
retriever



VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000025A289837D0>)

In [38]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, "langchain_docs", "useful for when you need to find information about langchain")

In [39]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
arxiv = ArxivAPIWrapper()
arxiv_tool = ArxivQueryRun(api_wrapper=arxiv)

In [40]:
tools = [wiki, arxiv_tool, retriever_tool]

In [41]:
tools

[WikipediaQueryRun(verbose=True, api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\Practice\\langchain-practice\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(),
 Tool(name='langchain_docs', description='useful for when you need to find information about langchain', args_schema=<class 'langchain_core.tools.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x00000259C3681120>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000025A289837D0>), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x00000259C36811C0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_commu

In [42]:
from dotenv import load_dotenv
load_dotenv()
import os
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0, model="gpt-4", max_retries=3)

In [43]:
from langchain import hub
prompt = hub.pull('hwchase17/openai-functions-agent')
prompt.messages

d:\Practice\langchain-practice\venv\Lib\site-packages\langchain\hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [44]:
from langchain.agents import  create_openai_tools_agent
agent = create_openai_tools_agent(llm, tools, prompt)

In [45]:
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a165

In [48]:
agent_executor.invoke({"input":"What is LangChain? Summarize it in a few sentences."})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'LangChain'}`


Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChainLangChain is a software framework designed to facilitate the integration of large language models (LLMs) into applications. It serves as a language model integration framework, providing a streamlined process for incorporating LLMs into various applications.

> Finished chain.


{'input': 'What is LangChain? Summarize it in a few sentences.',
 'output': 'LangChain is a software framework designed to facilitate the integration of large language models (LLMs) into applications. It serves as a language model integration framework, providing a streamlined process for incorporating LLMs into various applications.'}

In [49]:
agent_executor.invoke({"input":"What is n8n and how can it be used with LangChain?"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'n8n'}`


Page: N8n
Summary: n8n (pronounced "n-eight-n") is a workflow automation platform developed by n8n GmbH, headquartered in Berlin, Germany. The software provides a visual, node-based editor for buildin
Invoking: `langchain_docs` with `{'query': 'n8n integration'}`


​2. Authentication Setup
Open Agent Platform uses Supabase for authentication by default.
1Create a Supabase ProjectCreate a new project in Supabase.2Configure Environment VariablesSet the following environment variables inside the apps/web/ directory:You can find these Next.js-specific variables in the “Connect” window in the Supabase dashboard, under “App Frameworks”CopyNEXT_PUBLIC_SUPABASE_URL="<your supabase url>"
NEXT_PUBLIC_SUPABASE_ANON_KEY="<your supabase anon key>"
3Enable Authentication ProvidersEnable Google authentication in your Supabase project, or set NEXT_PUBLIC_GOOGLE_AUTH_DISABLED=true in your environment variables to d

{'input': 'What is n8n and how can it be used with LangChain?',
 'output': 'n8n (pronounced "n-eight-n") is a workflow automation platform developed by n8n GmbH, headquartered in Berlin, Germany. The software provides a visual, node-based editor for building workflows.\n\nAs for its integration with LangChain, the documentation does not provide specific details on how n8n can be used with LangChain. However, given that n8n is a workflow automation tool, it could potentially be used to automate various tasks and processes within LangChain, such as data processing, task scheduling, and more.\n\nFor example, you could set up a workflow in n8n to automatically process and analyze new data as it comes into LangChain, or to schedule regular updates and maintenance tasks. This could help to streamline your work with LangChain and make your processes more efficient.\n\nPlease note that this is a general suggestion and the actual integration process would depend on the specific requirements and